Consulta o banco de dados e salva em csv cada tabela

In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import timedelta
import os

save_path = r"endereço de salvamento"
# Database connection
engine = create_engine(
    "endereço base de dados", 
    echo=False
)
#lista de nomes possíveis para as colunas
nomes_coluna = ["Impp", "IV Point[3f612]-i", "IV Point[a9112]-i"]

# Defina o limite de tempo
tempo_limite = pd.Timestamp.now().floor('s') - timedelta(minutes=33)
print(tempo_limite)

# Query to get table names
table_query = text(
    "SELECT tablename FROM pg_tables WHERE tablename LIKE '%_POS%' AND tablename NOT LIKE '%OLD%' AND schemaname = 'public'"
)

with engine.connect() as conn:
    # Execute the query to get table names
    table_names = conn.execute(table_query).fetchall()

    # Iterate through table names and export each to a CSV file
    for table_name in table_names:
        print(table_name)
        table_name = table_name[0]  # Get the table name string from the result

        # Create a flag to check if any column was found
        column_found = False

        # Iterate through possible column names
        for col_name in nomes_coluna:
            # Check if the column exists in the table
            query = text(f'SELECT column_name FROM information_schema.columns WHERE table_name = :table_name AND column_name = :col_name')
            result = conn.execute(query, {'table_name': table_name, 'col_name': col_name}).fetchone()
            
            if result:  # If the column exists
                query = text(f'SELECT "TIMESTAMP", "{col_name}" FROM "{table_name}"  WHERE "TIMESTAMP" >= :tempo_limite')
                df = pd.read_sql(query, conn, params={'tempo_limite': tempo_limite})  # Read the table into a DataFrame
                csv_file_name = f"\\{table_name.replace('/', '_')}-{col_name}.csv"
                final_save_path = save_path + csv_file_name        

                df.to_csv(final_save_path, index=False)  # Save DataFrame to CSV
                column_found = True  # Set flag to True

        # Check if no columns were found
        if not column_found:
            print(f"No matching columns found in table {table_name}")

print("CSV files have been created.")


2024-05-13 08:49:00
('A231107A-023/M_POS16_2023-12-21',)
('A240305F-732_POS01_2024-04-20',)
('A240305F-351_POS02_2024-04-20',)
('A240312F-041_POS04_2024-04-20',)
('A240312F-062_POS06_2024-04-20',)
('A240320F-028_POS08_2024-04-20',)
('A240305F-731_POS03_2024-04-20',)
('A240312F-061_POS05_2024-04-20',)
('A240305F-111_POS10_2024-04-20',)
('A240320F-006_POS12_2024-04-20',)
('A240305F-062_POS14_2024-04-20',)
('A240305F-061_POS17_2024-04-20',)
('A240312F-002_POS18_2024-04-20',)
('A240312F-151_POS07_2024-04-20',)
('A240312F-132_POS09_2024-04-20',)
('A240320F-004_POS11_2024-04-20',)
('A240305F-073_POS13_2024-04-20',)
('A240312F-001_POS15_2024-04-20',)
CSV files have been created.
